## Widgets

In [0]:
# Crear un widget para el catálogo
dbutils.widgets.text("catalog", "SUSTITUIR")

## Cálculos en Gold
Leer de la tabla h_transactions de Silver, hacer calculos y escribir amount_by_merchantCategory en Gold

In [0]:
from pyspark.sql.functions import round

# Leer el catálogo del widget
catalog = dbutils.widgets.get("catalog")

# Leer la tabla h_transactions desde el catálogo de Unity
df = spark.read.table(f"{catalog}.silver.h_transactions")

# Convertir el campo 'amount' a tipo numérico
df = df.withColumn("amount", df["amount"].cast("double"))

# Calcular el importe total por merchantCategory
total_amount_by_market = df.groupBy("merchantCategory").sum("amount").withColumnRenamed("sum(amount)", "total_amount")
total_amount_by_market = total_amount_by_market.withColumn("total_amount", round("total_amount", 2))

# Crear el esquema si no existe
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")

# Guardar el resultado en la capa gold
total_amount_by_market.write.mode("overwrite").saveAsTable(f"{catalog}.gold.amount_by_merchantCategory")

# Mostrar el resultado
display(total_amount_by_market)